<h1>Transfer Learning - ResNet152<h1>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/BTech_Project/preprocessed_v2/Training'
test_dir = '/content/drive/MyDrive/Colab Notebooks/BTech_Project/preprocessed_v2/Testing'

In [ ]:
# Image augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    validation_split=0.15  # 15% validation data split
)

# No augmentation for validation and test sets
val_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.15)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Adjust target size for ResNet
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 4687 images belonging to 4 classes.
Found 826 images belonging to 4 classes.
Found 1212 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.regularizers import l2

# Load the pre-trained ResNet152 base model without top layers
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global pooling layer
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)  # Dense layer with L2 regularization
x = Dropout(0.5)(x)  # Dropout layer for regularization
x = Dense(512, activation='relu')(x)  # Another dense layer
x = Dropout(0.5)(x)  # Dropout layer for regularization
x = Dense(4, activation='softmax')(x)  # Output layer for 4 classes

# Combine base model and custom layers into one model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers initially for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with an Adam optimizer
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks to adjust learning rate and prevent overfitting
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

# Save the initial trained model
model.save('ResNet152_initial.keras')

Epoch 1/15
147/147 [==============================] - 1104s 7s/step - loss: 2.3001 - accuracy: 0.3358 - val_loss: 1.7982 - val_accuracy: 0.3571 - lr: 1.0000e-04
Epoch 2/15
147/147 [==============================] - 187s 1s/step - loss: 1.5640 - accuracy: 0.4318 - val_loss: 1.4264 - val_accuracy: 0.4492 - lr: 1.0000e-04
Epoch 3/15
147/147 [==============================] - 189s 1s/step - loss: 1.2967 - accuracy: 0.4536 - val_loss: 1.2696 - val_accuracy: 0.5073 - lr: 1.0000e-04
Epoch 4/15
147/147 [==============================] - 186s 1s/step - loss: 1.1969 - accuracy: 0.4773 - val_loss: 1.2425 - val_accuracy: 0.4782 - lr: 1.0000e-04
Epoch 5/15
147/147 [==============================] - 188s 1s/step - loss: 1.1605 - accuracy: 0.5029 - val_loss: 1.2051 - val_accuracy: 0.5327 - lr: 1.0000e-04
Epoch 6/15
147/147 [==============================] - 188s 1s/step - loss: 1.1283 - accuracy: 0.5121 - val_loss: 1.1766 - val_accuracy: 0.4891 - lr: 1.0000e-04
Epoch 7/15
147/147 [===================

In [ ]:
# Unfreeze the last few layers of the base model for fine-tuning
for layer in base_model.layers[-90:]:
    layer.trainable = True

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

# Save the fine-tuned model
model.save('ResNet152_finetuned.keras')

Epoch 1/15
147/147 [==============================] - 275s 2s/step - loss: 1.0670 - accuracy: 0.5374 - val_loss: 1.3267 - val_accuracy: 0.3317 - lr: 1.0000e-05
Epoch 2/15
147/147 [==============================] - 257s 2s/step - loss: 0.9930 - accuracy: 0.5788 - val_loss: 1.0951 - val_accuracy: 0.5508 - lr: 1.0000e-05
Epoch 3/15
147/147 [==============================] - 258s 2s/step - loss: 0.9719 - accuracy: 0.5985 - val_loss: 1.1002 - val_accuracy: 0.5593 - lr: 1.0000e-05
Epoch 4/15
147/147 [==============================] - 257s 2s/step - loss: 0.9472 - accuracy: 0.6196 - val_loss: 1.2588 - val_accuracy: 0.5048 - lr: 1.0000e-05
Epoch 5/15
147/147 [==============================] - 257s 2s/step - loss: 0.9225 - accuracy: 0.6275 - val_loss: 1.1469 - val_accuracy: 0.5472 - lr: 1.0000e-05
Epoch 6/15
117/147 [======================>.......] - ETA: 46s - loss: 0.9071 - accuracy: 0.6341

In [ ]:
print(f"Final fine-tune training accuracy: {fine_tune_history.history['accuracy'][-1]:.4f}")
print(f"Final fine-tune validation accuracy: {fine_tune_history.history['val_accuracy'][-1]:.4f}")

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

# Predictions for test data
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
#classification report
class_labels = list(test_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

In [ ]:
#confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()